In [1]:
import cv2
import numpy as np
import sys
import time
from imutils.object_detection import non_max_suppression
from imutils import paths
import imutils
import random
import os, copy

In [2]:
global img
global point1, point2
global g_rect
global key_list

In [3]:
# Retrieved：https://pyimagesearch.com/2015/02/16/faster-non-maximum-suppression-python/
def NMS(boxes, threshold):
    if len(boxes) == 0:
        return []
    
    boxes = np.array(boxes).astype("float")

    x_tl = boxes[:,0]  # Abscissa of top left corner vertex
    y_tl = boxes[:,1]  # Ordinate of top left corner vertex
    w = boxes[:,2]  # Width of bounding box
    h = boxes[:,3]  # Height of bounding box
    x_br = x_tl + w   
    y_br = y_tl + h   
    
    area = (w + 1) * (h + 1)  # Calculate the area of each rectangular box, add 1 to make the IOU match without 0
    temp = []
    
    idxs = np.argsort(h)  #Sort the elements in h from small to large and return the subscript of each element in h
    
    while len(idxs) > 0:
        last = len(idxs) - 1
        i = idxs[last]
        temp.append(i)   
        
        x_tl_m = np.maximum(x_tl[i], x_tl[idxs[:last]])   # Compare the abscissas of the upper left corner of other rectangles
        y_tl_m = np.maximum(y_tl[i], y_tl[idxs[:last]])   # Pairwise comparison of the ordinates of the upper left corner of other rectangles

        x_br_m = np.minimum(x_br[i], x_br[idxs[:last]]) 
        y_br_m = np.minimum(y_br[i], y_br[idxs[:last]])

        w_m = np.maximum(0, x_br_m - x_tl_m + 1)  # Calculate the width of the bounding box
        h_m = np.maximum(0, y_br_m - y_tl_m + 1)  # Calculate the height of the bounding box
         
        over = (w_m * h_m) / area[idxs[:last]]  # Calculates the ratio of the overlapping rectangle area to the area in area
        
        idxs = np.delete(idxs, np.concatenate(([last],   
            np.where(over > threshold)[0])))    # remove overlapping rectangles

    return boxes[temp].astype("int")



In [4]:
# Retrieved from: https://dandelioncloud.cn/article/details/1504109922504765442
def on_mouse(event, x, y, flags, param):
    global img, point1, point2, g_rect
    img2 = img.copy()
    count = 0
    if event == cv2.EVENT_LBUTTONDOWN:   
        point1 = (x, y)
        cv2.circle(img2, point1, 10, (0, 255, 0), 5)
        cv2.imshow('pedestrians', img2)
    elif event == cv2.EVENT_MOUSEMOVE and (flags & cv2.EVENT_FLAG_LBUTTON):   
        cv2.rectangle(img2, point1, (x, y), (255, 0, 0), thickness=2)
        cv2.imshow('pedestrians', img2)
    elif event == cv2.EVENT_LBUTTONUP:   
        point2 = (x, y)
        cv2.rectangle(img2, point1, point2, (0, 0, 255), thickness=2)
        cv2.imshow('pedestrians', img2)
        if point1 != point2:
            min_x = min(point1[0], point2[0])
            min_y = min(point1[1], point2[1])
            width = abs(point1[0] - point2[0])
            height = abs(point1[1] - point2[1])
            g_rect = [min_x, min_y, width, height]
            cut_img = img[min_y:min_y + height, min_x:min_x + width]
            # Traverse the list that stores all the people in the current frame, obtain the coordinates of 
            # the center point of the corresponding person in the dictionary, and determine whether the 
            # coordinates of the center point are within the boundary of the cut image
            for key in key_list:
                if min_x<=dic_res[key][-1][0]<= min_x + width and min_y<=dic_res[key][-1][1]<= min_y + height: 
                    count += 1
            cv2.putText(cut_img,   f'{count}'   ,(20,40), cv2.FONT_HERSHEY_COMPLEX,   1,  (0,255,0),   2)
            cv2.imshow('pedestrians_cut', cut_img)

In [5]:
def get_key(dic,value):
    return [k for (k,v)in dic.items() if v == value]

In [6]:
def calculate_distance(p1,p2):
    x1,y1 = p1
    x2,y2 = p2
    return pow(pow(x1-x2,2)+pow(y1-y2,2),0.5)

In [7]:
def get_random_color():
    r = lambda: random.randint(0,255)
    return (r(),r(),r())

In [8]:
def svmdetectpersons(img):
    hog=cv2.HOGDescriptor()
    hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())
    person, w = hog.detectMultiScale(img, winStride=(4, 4), padding=(8, 8),scale=1.05,useMeanshiftGrouping=False)
#     person, w = hog.detectMultiScale(img,hitThreshold = -0.2)
    return person

In [9]:
def get_center(image, person):
    x, y, w, h = person
    return (x+w//2, y+h//2)

In [10]:
def draw_person(image, person,color):
    x, y, w, h = person
    cv2.rectangle(image, (x, y), (x + w, y + h), color, 4)

In [11]:
def absdiff_demo(img_1, img_2, sThre):
    gray_image_1 = cv2.cvtColor(img_1, cv2.COLOR_BGR2GRAY)
    gray_image_1 = cv2.GaussianBlur(gray_image_1,(5,5),0)    # GaussianBlur 
#     gray_image_1 = cv2.bilateralFilter(gray_image_1,-1,15,10)  
    gray_image_2 = cv2.cvtColor(img_2, cv2.COLOR_BGR2GRAY)   
    gray_image_2 = cv2.GaussianBlur(gray_image_2,(5,5),0)
    d_frame = cv2.absdiff(gray_image_1,gray_image_2)         # Frame difference
    ret, d_frame = cv2.threshold(d_frame, sThre, 255, cv2.THRESH_BINARY)  # Binarization
    return d_frame

In [12]:
# Retrieved from: https://www.tutorialspoint.com/program-to-check-two-rectangular-overlaps-or-not-in-python
# judge two people in group or not
def booleanOverlap(nparray1, nparray2):
    nparray3 = copy.deepcopy(nparray1)
    nparray4 = copy.deepcopy(nparray2)
    nparray3[2] = nparray3[0] + nparray3[2]
    nparray3[3] = nparray3[1] + nparray3[3]
    nparray4[2] = nparray4[0] + nparray4[2]
    nparray4[3] = nparray4[1] + nparray4[3]
    #Nparray3[0] the left abscissa of object one. Nparray4[2] right abscissa of object two
    #Nparray3[2] the right abscissa of object one. Nparray4[0] left abscissa of object two
    if (nparray3[0]>=nparray4[2]) or (nparray3[2]<=nparray4[0]) or (nparray3[3]<=nparray4[1]) or (nparray3[1]>=nparray4[3]):
        return False
    else:
        return True

# return number pedestrians in group working and walking alone
def aloneOrGroup(filtered, frame_copy):
    alone_count = 0
    filtered_dict = {} #(coordinates:flag)
    for i in range(len(filtered)):
        flag = 0 #alone
#         alone_count = 0
        for j in range(len(filtered)):
            if i==j: continue
            else:
                # Walking together means have intersection and have almost same size
                # The width ratio is between 0.75 and 1.33, indicating that the two objects are about the same size. 
                # If they coincide, it means that they are groups together
                if booleanOverlap(filtered[i], filtered[j]) == True and 0.75*filtered[j][3]<=filtered[i][3]<=1.33*filtered[j][3]:
                    flag = 1 #group
                    break
        center_point = get_center(frame_copy,filtered[i])
        filtered_dict[str(list(center_point))] = flag

        if flag == 0:
            alone_count += 1

    # print(len(filtered))
    # print(alone_count, len(filtered)-alone_count)
    return alone_count, len(filtered)-alone_count, filtered_dict

### change the path here

In [13]:
# img_1 = cv2.imread('./train/STEP-ICCV21-02/000001.jpg')
# img_2 = cv2.imread('./train/STEP-ICCV21-02/000002.jpg')
# './test/STEP-ICCV21-01'
# './test/STEP-ICCV21-07'
# './train/STEP-ICCV21-02'
# './train/STEP-ICCV21-09'
# input the path here
path = './test/STEP-ICCV21-01'

In [14]:
train_loss_list = []
val_loss_list = []
loss_file_name = []
files = os.listdir(path)  # Use listdir to read all files
for i in files:
    loss_file_name.append(path +'/' + i)
    loss_file_name.sort(key=lambda x: x)  # Sort by numeric characters
file_path = loss_file_name 
sThre = 40  

In [16]:
dic_res = {}
key_people = []
filtered_dict_prev = {}

for img_path in file_path:
    key_list = []
    count_person_enter = 0
    count_person_leave = 0
    frame = cv2.imread(img_path)
    if cv2.waitKey(1) & 0xFF == 13:
        cv2.destroyAllWindows()
        break
    frame_copy = frame.copy()
    # Preliminary screening of pedestrians by three-frame difference method
#     temp01 = absdiff_demo(img_1, img_2, sThre)
#     temp12 = absdiff_demo(frame, img_2, sThre)
#     img_1 = img_2
#     img_2 = frame
#     binary = cv2.bitwise_and(temp01,temp12)
#     kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
#     erode = cv2.erode(binary,kernel,iterations=1) #erode
#     dilate = cv2.dilate(erode,kernel,iterations=1) #dilate
#     dilate = cv2.dilate(dilate,kernel,iterations=1) #dilate
#     erode = cv2.erode(dilate,kernel,iterations=1) #erode
    person = svmdetectpersons(frame)  
    filtered = []
    for item in NMS(person, threshold=0.8):
        if get_center(frame_copy,item)[1]>400:
            filtered.append(item)
    t = aloneOrGroup(filtered, frame_copy)
    if filtered_dict_prev == {}:
        filtered_dict_prev = copy.deepcopy(t[2])

    for p in filtered:    #  [[1,2,3,4],[2,3,4,5],[3,4,5,6],[6,7,8,9]]
        flag_out = 0
        # Get image center point
        center_point = get_center(frame_copy,p)
        # iterate the dictionary
        for keys, values in dic_res.items():
            # Calculate the distance between the new point and the point in the original dictionary
            # values[-1] means only the closest comparison with all points
            if calculate_distance(center_point,values[-1]) < 45:   ## 
                # If the distance is less than the threshold of 45, 
                # the two points are considered to be one target, 
                # and the center point is added to the corresponding position in the dictionary
                if str(list(values[-1])) in filtered_dict_prev.keys() and str(list(center_point)) in t[2].keys():
                    if filtered_dict_prev[str(list(values[-1]))] == 0 and t[2][str(list(center_point))] == 1:#group formation/yellow
                        cv2.circle(frame_copy, center_point, 30, (0, 255, 255), 5)
                    if filtered_dict_prev[str(list(values[-1]))] == 1 and t[2][str(list(center_point))] == 0:#group destruction/blue
                        cv2.circle(frame_copy, center_point, 30, (255, 0, 0), 5)
                filtered_dict_prev = copy.deepcopy(t[2])
                new_list = values
                new_list.append(center_point)
                # new_list[-1][0] > frame_copy.shape[1] - 500 ：The center point is beyond the frame, and the distance from the right border of the frame to the right border of the image is 500
                # new_list[-1][0] < 400 ：The center point is beyond the frame, and the distance from the left border of the frame to the left border of the image is 400
                # new_list[-1][1] < 0 ：The center point is beyond the frame, and the distance from the upper border of the frame to the upper border of the image is 50
                # new_list[-1][1] > frame_copy.shape[0] - 400 ：The center point is beyond the frame, and the distance from the lower border of the frame to the lower border of the image is 400
                if new_list[-1][0] > frame_copy.shape[1] - 500 or new_list[-1][0] < 400 \
                    or new_list[-1][1] < 0 or new_list[-1][1] > frame_copy.shape[0] - 400:
#                         cv2.circle(frame_copy, center_point, 10, (0, 255, 0), 5)
#                         count_person_leave += 1
                    if calculate_distance(new_list[-1],(frame_copy.shape[1]//2,frame_copy.shape[0]//2)) > \
                        calculate_distance(new_list[-2],(frame_copy.shape[1]//2,frame_copy.shape[0]//2)):
                        cv2.circle(frame_copy, center_point, 10, (0, 255, 0), 5)
                        count_person_leave += 1

                if (new_list[-2][0] > frame_copy.shape[1] - 500 or new_list[-2][0] < 400 \
                    or new_list[-2][1] < 0  or new_list[-2][1] > frame_copy.shape[0] - 400) and \
                    (400 < center_point[0] < frame_copy.shape[1] - 500  \
                    and  0  < center_point[1] < frame_copy.shape[0] - 400):
                    cv2.circle(frame_copy, center_point, 30, (0, 0, 255), 5)
                    count_person_enter += 1
                # Rewrite the values of the corresponding keys
                dic_res[keys] = new_list       #   dic_res[A] = [[1,2],[2,3]]   # [2,3]
                draw_person(frame_copy,p,keys)
                key_list.append(keys)
                for i in range(len(new_list)-1):
                    cv2.line(frame_copy, new_list[i], new_list[i+1], keys, thickness=2)
                flag_out = 1
                break
        # If the old center point is not found within the threshold distance from the new center point after traversing the dictionary, 
        # the new center point is considered to be the center point of the new character, and a new person is added in dic_res
        if flag_out != 1:
            dic_res[get_random_color()] = [center_point]
            keys = get_key(dic_res,[center_point])[0]
            draw_person(frame_copy,p,keys)
            key_list.append(keys)
            if center_point[0] < frame_copy.shape[1] - 400 or center_point[0] > 400 \
            or center_point[1] > 0  or center_point[1] < frame_copy.shape[0] - 400:
                cv2.circle(frame_copy, center_point, 30, (0, 0, 255), 5)
                count_person_enter += 1
    key_people = key_list
    cv2.putText(frame_copy,   f'People number:{len(filtered)}'   ,(0,40), cv2.FONT_HERSHEY_COMPLEX,   1,  (0,255,0),   2)
    cv2.putText(frame_copy, f'Number of People walking alone:{t[0]}',(0,80), cv2.FONT_HERSHEY_COMPLEX,   1,  (0,0,255),   2)
    cv2.putText(frame_copy, f'Number of People walking in group:{t[1]}',(0,120), cv2.FONT_HERSHEY_COMPLEX,   1,  (0,0,255),   2)
    cv2.putText(frame_copy, f'Number of People enter the scene:{count_person_enter}',(0,160), cv2.FONT_HERSHEY_COMPLEX,   1,  (0,0,255),   2)
    cv2.putText(frame_copy, f'Number of People leave the scene:{count_person_leave}',(0,200), cv2.FONT_HERSHEY_COMPLEX,   1,  (0,255,0),   2)
    img = frame_copy
    cv2.namedWindow('pedestrians')
    cv2.startWindowThread()
    cv2.imshow('pedestrians', frame_copy)
    cv2.setMouseCallback('pedestrians', on_mouse)

    # Press "Enter" to exit the window
    if cv2.waitKey(1) & 0xFF == 13:
        break

cv2.destroyAllWindows()